Bartlett's test, developed by [Maurice Stevenson Bartlett](https://en.wikipedia.org/wiki/M._S._Bartlett), is a statistical procedure for testing if $k$ population samples have equal variances. Equality of variances in population samples is assumed in commonly used comparison of means tests, such as [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test) and [analysis of variance](https://en.wikipedia.org/wiki/Analysis_of_variance). Therefore, a procedure such as Bartlett's test can be conducted to accept or reject the assumption of equal variances across group samples. 

Levene's test is an alternative to Barlett's test and is less sensitive to non-normal samples. Thus, Levene's test is generally preferred in most cases, particularly when the underlying distribution of the samples is known. Equality of variances is also known as [homoscedasticity](https://en.wikipedia.org/wiki/Homoscedasticity) or homogeneity of variances.

Bartlett's test statistic, denoted $\chi^2$ (also sometimes denoted as $T$), is approximately chi-square distributed with $k - 1$ degrees of freedom, where $k$
is the number of sample groups. The chi-square approximation does not hold sufficiently when the sample size of
a group is $n_i > 5$. The test statistic is defined as:

$$ \chi^2 = \frac{(n - k) \ln(S^2_p) - \sum^k_{i=1} (n_i - 1) \ln(S^2_i)}{1 + \frac{1}{3(k - 1)} \left(\sum^k_{i=1} (\frac{1}{n_i - 1}) - \frac{1}{n - k} \right)} $$

where, 

* $n$ is the total number of samples across all groups
* $k$ is the number of groups
* $S^2_i$ are the sample variances.

$S^2_p$, the pooled estimate of the samples' variance, is defined as:

$$ S^2_p = \frac{1}{n - k} \sum_i (n_i - 1) S^2_i $$

### Bartlett's Test in Python

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
import numpy_indexed as npi

The [`PlantGrowth`](https://vincentarelbundock.github.io/Rdatasets/doc/datasets/PlantGrowth.html) dataset is available in [R](https://www.r-project.org/) as part of its standard datasets and can also be downloaded [here](https://vincentarelbundock.github.io/Rdatasets/csv/datasets/PlantGrowth.csv). After downloading the data, we load it into memory with pandas' [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function. Once the data is loaded, we transform the resulting `DataFrame` into a [`numpy`](https://numpy.org/) array with the [`.to_numpy`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html) method. The first three rows of the dataset are then printed to get a sense of what the data contains. We also confirm there are indeed three sample groups in the data using numpy's [`unique`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html) function.

In [2]:
plants = pd.read_csv('../../data/PlantGrowth.csv')
plants = plants.to_numpy()

print(plants[:3])
print(list(np.unique(plants[:,2])))

[[1 4.17 'ctrl']
 [2 5.58 'ctrl']
 [3 5.18 'ctrl']]
['ctrl', 'trt1', 'trt2']


Implementing the test procedure in Python is comparatively straightforward. First, we require the total number of samples, $n$, and the number of sample groups, $k$. The number of samples can be found by indexing the first value returned from the `plants` array [`.shape`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html) method, while the number of unique groups is obtained by taking the length of the array returned by the `unique` function. We also require the total number of samples within each group and the group's variance, which we compute by grouping the `plants` array by `len` and `numpy`'s [`var`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.var.html) functions. The [`numpy-indexed`](https://pypi.org/project/numpy-indexed/) library and its `group_by` function are convenient for performing this grouping operation on `numpy` arrays.

In [3]:
n = plants.shape[0]
k = len(np.unique(plants[:,2]))

group_n = np.array([i for _, i in npi.group_by(plants[:, 2], plants[:, 1], len)])
group_variance = np.array([i for _, i in npi.group_by(plants[:, 2], plants[:, 1], np.var)])

The sample pooled variance, $S^2_p$, is then computed.

In [4]:
pool_var = 1 / (n - k) * np.sum((group_n - 1) * group_variance)

As the Bartlett test statistic equation is rather hefty, we split the computation into two variables, the numerator and denominator of the test statistic, to help keep the code easier to read. The numerator and denominator are then divided, which returns the computed test statistic.

In [5]:
x2_num = (n - k) * np.log(pool_var) - np.sum((group_n - 1) * np.log(group_variance))
x2_den = 1 + 1 / (3 * (k - 1)) * (np.sum(1 / (group_n - 1)) - 1 / (n - k))

x2 = x2_num / x2_den
x2

2.8785737872360935

The Bartlett test statistic, $\chi^2$, is approximately $2.279$. To find the associated p-value of the test statistic, we using the `.cdf` method from `scipy.stats`'s [`chi2`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html) variable with $k - 1$ degrees of freedom.

In [6]:
p = 1 - chi2.cdf(x2, k - 1)
p

0.23709677363455817

The reported p-value is over $0.05$. Therefore we accept the null hypothesis that the group variances are equal.

### References

[Snedecor, G. W., & Cochran, W. G. (1989). Statistical methods (8th ed.). Ames, IA: Iowa State University Press.](https://amzn.to/2UwYNCO)

[Wikipedia contributors. "Bartlett's test." Wikipedia, The Free Encyclopedia. Wikipedia, The Free Encyclopedia, 17 Feb. 2020. Web. 29 Mar. 2020.](https://en.wikipedia.org/wiki/Bartlett%27s_test)

https://www.itl.nist.gov/div898/handbook/eda/section3/eda357.htm